In [14]:
!pip install transformers gradio sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 10.8 MB/s eta 0:00:00


In [15]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline, AutoModelForCausalLM

# Load ClinicalBERT
bert_model_name = "emilyalsentzer/Bio_ClinicalBERT"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
bert_model = AutoModelForQuestionAnswering.from_pretrained(bert_model_name)
qa_pipeline = pipeline("question-answering", model=bert_model, tokenizer=bert_tokenizer)

# Load BioGPT
biogpt_model_name = "microsoft/BioGPT"
biogpt_tokenizer = AutoTokenizer.from_pretrained(biogpt_model_name)
biogpt_model = AutoModelForCausalLM.from_pretrained(biogpt_model_name)

# Combine both models for comparison
def compare_models(context, question):
    # ClinicalBERT answer
    bert_result = qa_pipeline({
        "context": context,
        "question": question
    })["answer"]

    # BioGPT generated answer
    prompt = f"Context: {context}\nQuestion: {question}\nAnswer:"
    inputs = biogpt_tokenizer(prompt, return_tensors="pt")
    output_ids = biogpt_model.generate(
        **inputs,
        max_length=100,
        do_sample=True,
        top_p=0.95,
        temperature=0.7,
        num_return_sequences=1
    )
    biogpt_result = biogpt_tokenizer.decode(output_ids[0], skip_special_tokens=True)
    biogpt_result = biogpt_result.split("Answer:")[-1].strip()

    return bert_result, biogpt_result

# Gradio interface
gr.Interface(
    fn=compare_models,
    inputs=[
        gr.Textbox(lines=8, label="📝 Clinical Context"),
        gr.Textbox(lines=1, label="❓Question")
    ],
    outputs=[
        gr.Textbox(label="🧠 ClinicalBERT Answer"),
        gr.Textbox(label="🧬 BioGPT Answer")
    ],
    title="Dual Medical QA: ClinicalBERT vs BioGPT",
    description="Compare extractive vs generative answers for the same medical input using two pretrained models."
).launch()


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


pytorch_model.bin:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://eddbfe52c157cec332.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
